# Søk i data-norge ved hjelp av LLM


## Del 1: Oppsett

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

Oppsett-biten av denne notebooken trenger kun å gjøres en gang. Første steg er å sette opp konfig for GCP prosjekt. Denne er nå satt opp mot TDS sitt prosjekt i Entur sin GCP.

In [2]:
project_id = os.getenv('GCP_PROJECT_ID')
region = os.getenv('GCP_REGION')

Vi må også sette opp konfig for Cloud SQL. Vi har en instance i prosjektet som kjører en PostgreSQL server.

In [3]:
# Cloud SQL info
instance_name = os.getenv('SQL_INSTANCE_ID')
database_user = os.getenv('SQL_DB_USER')
database_password = os.getenv('SQL_PASSWORD')
database_name = "fdk-v1"

Sett opp søke-variabler. Disse variablene styrer søket i PostgreSQL databasen. Disse kan også endres per søk, se lenger ned i notebooken for eksempel.

Num_matches er max antall treff returnert fra query.

Similarity_threshold er hvor likt et datasett må matche søk for å returneres. Må være en verdi mellom 0 og 1.

In [4]:
# Query info
num_matches = 10
similarity_threshold = 0.5

Importer nødvendige pakker. Enkelte av disse må først installeres i python-miljøet

In [ ]:
!pip install langchain
!pip install langchain_google_vertexai
!pip install openai
!pip install cohere
!pip install tiktoken
!pip install asyncpg
!pip install pgvector
!pip install cloud-sql-python-connector

In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from google.cloud.sql.connector import Connector
from pgvector.asyncpg import register_vector
from google.cloud import aiplatform
import pandas as pd
import asyncio
import asyncpg



Nå må vi initialisere prosjektet for bruk av AI.


In [7]:
aiplatform.init(project=f"{project_id}", location=f"{region}")

Dette er vår hovedfunksjon for å finne liknende datasett ut ifra en query. Vi bruker et kosinus-søk for å finne liknende datasett. Disse datasettene blir senere matet inn i en LLM som bruker de til å besvare spørsmålet på en fornuftig måte.

In [8]:
async def main(qe):
    matches = []
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )
        await register_vector(conn)

        # Use cosine similarity search to find the top five products
        # that are most closely related to the input query.

        results = await conn.fetch("""
                     WITH vector_matches AS (
                             SELECT id,
                                    1 - (embedding <=> $1) AS similarity
                             FROM dataset_embeddings
                             WHERE 1 - (embedding <=> $1) > $2
                             ORDER BY similarity DESC
                             LIMIT $3
                     )
                     SELECT id,
                            summary
                     FROM datasets
                     WHERE id IN (SELECT id FROM vector_matches)
                     """,
                                   qe, similarity_threshold, num_matches)

        if len(results) == 0:
            print("Dessverre fant vi ikke noe matchende datasett.")

        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(r["summary"])

        await conn.close()
        return matches

Under er instruksjonene vi gir til LLM modellen for å få gode svar. Du må gjerne eksperimentere litt for å prøve å få bedre svar.

User query her er spørsmålet som stilles til LLM-en.

Description vil være en oppsummering av alle datasettene som returneres fra søket mot PostgreSQL.

In [58]:
map_prompt_template = """


              You will be given a detailed description of a published dataset in norwegian.
              This description is enclosed in triple backticks (```).
              Using this description only, extract the title of the dataset,
              the category and its most useful features.

              ```{text}```
              SUMMARY:
              """

combine_prompt_template = """
                You will be given a detailed description of different datasets in norwegian
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                Select the datasets that is most relevant to answer the question.
                Using that selected dataset description, answer the following
                question in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the title of the dataset, and why it matches the question posed by the user.
                Structure your answer to be easily readable by a human person.
                Do not answer in markdown format.
                If no datasets are given, assume there are no datasets matching the
                question, and explain that the data might not exist.
                Try to refer to the user directly.
                Give the answer in Norwegian.

                Description:
                ```{text}```


                Question:
                ``{user_query}``


                Answer:
                """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
combine_prompt = PromptTemplate(template=combine_prompt_template, input_variables=["text", "user_query"])

For å kunne koble deg til GCP-projektet vårt må du først autentisere deg gjennom Google. Dette gjøres via en enkel innlogging som startes automatisk når koden under kjøres. Google-brukeren din må ha tilgang til GCP-prosjektet

```
ent-data-fdkllm-ext-dev
```



In [10]:
from google.colab import auth
auth.authenticate_user()

Dette er siste del av oppsettet. Vi starter opp Vertex AI som er LLM-modellen vi bruker.

In [60]:
llm = VertexAI(model_name="text-bison", max_output_tokens=1000)
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt,
                             combine_prompt=combine_prompt)

## Del 2: Interaksjon mot modellen

Nå skal vi endelig gjøre spørringer mot modellen vår.
Alt du trenger å gjøre er å oppgi et spørsmål eller beskrivelse under 'user_query' og kjøre de tre neste kodeblokkene i sekvens. Du må gjerne også eksperimentere med de to andre variablene, men disse må ikke endres hvis du ikke føler for det.

In [62]:
# @markdown Vennligst fyll ut spørsmål eller beskrivelse under.
# Please fill in these values.
user_query = "\"Har du data om tog i Bergen?\"\""  # @param {type:"string"}
num_matches = 5  # @param {type:"integer"}
similarity_threshold = 0.5  # @param {type:"number"}

# Quick input validations.
assert user_query, "⚠️ Please input a valid input search text"

# Generate vector embedding for the user query.
embeddings_service = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual@001")
qe = embeddings_service.embed_query(user_query)
matches = await main(qe)

docs = [Document(page_content=t) for t in matches]

answer = chain.invoke({
    'input_documents': docs,
    'user_query': user_query,
}, return_only_outputs=True)

print(f"QUESTION: {user_query}")
print(f"ANSWER: {answer['output_text']}")

/usr/local/lib/python3.10/dist-packages/google/cloud/sql/connector/refresh_utils.py:214: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


QUESTION: "Har du data om tog i Bergen?""
ANSWER:  Hei,

                Det finnes et datasett som heter "Trafikkdata fra Jernbaneverket" som kan være relevant for spørsmålet ditt. Dette datasettet inneholder rutetider og sanntidsinformasjon om togtrafikken i Norge, inkludert Bergen.

                Datasettet er kategorisert som "Ukjent", og de mest nyttige funksjonene er rutetider og sanntidsinformasjon om togtrafikken.

                Håper dette hjelper!
